# Surface age prediction using saved models

This code predict the SW age at 1 AU using the already saved models.

For itokawa, models saved in "Itokawa_Models" (These models were saved using the "Itokawa_model_training.ipynb")

For Eros, models saved in "Eros_Models" (These models were saved using the "Eros_model_training.ipynb").

In "Surface age correction", assign "a" 1.3 for Itokawa and 1.4 for Eros (Semi-major axis of Itokawa and Eros).

In [ ]:
!pip install tensorflow==2.17.1 keras==3.5.0

In [ ]:
!pip install scikeras==0.13.0

In [ ]:
!pip install scikit-learn==1.5.2

In [ ]:
!pip install numpy==1.26.4 joblib==1.4.2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import sklearn
import joblib
import os
import math

from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d
from keras.layers import Dense, Conv1D, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor,GradientBoostingRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.neighbors import KNeighborsRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import r2_score
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split

In [ ]:
#CNN model
def create_cnn_model_1(input_shape,learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding="same", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32, kernel_size=2, activation="relu", padding="same"))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(1, activation="relu"))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=["mean_absolute_error"])
    return model

In [ ]:
#Custom Keras Regressor class for CNN-based regression tasks.

class CNNRegressor(KerasRegressor):
    _estimator_type = "regressor"

    def __init__(self, input_shape, learning_rate=0.001, epochs=10, batch_size=32, verbose=0, **kwargs):
        super().__init__(**kwargs)
        self.input_shape = input_shape
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        #super().__init__(build_fn=self._keras_build_fn, **kwargs)

    def _keras_build_fn(self):
        return create_cnn_model_1(self.input_shape, self.learning_rate)

In [ ]:
# Define choices for H ion irradiation (H) and lasser irradiation (L)
choices = {'H': 3, 'L': 12}

input_file = "/file_with_the_reflectance.xlsx"  # Replace with your file name
input_data = pd.read_excel(input_file)

# Retain the first two columns as identifiers
identifiers = input_data.iloc[:, :3]  # First two columns
corrected = identifiers.copy()

# Separate features (X) and target (y), excluding the first two columns
X = input_data.iloc[:, 5:-1].to_numpy()  # Columns 3 to second-last as features
y = input_data.iloc[:, -1].copy()       # The last column (target)

# Initialize a DataFrame to store predictions for both H and L
all_predictions_combined = identifiers.copy()  # Start with identifiers

# Folder containing the saved models
models_folder = "/trained_models" # Replace with your folder name

for choice, adjustment_value in choices.items():
    # Adjust the target values based on the current choice
    y_adjusted = y + adjustment_value

    # Reconstruct the modified dataset for predictions
    X_modified = np.hstack([X, y_adjusted.values.reshape(-1, 1)])

    # Initialize a DataFrame to store predictions for this choice
    predictions = []
    for iteration in range(1, 31):
        model_filename = os.path.join(models_folder, f"En_Eros_{iteration}.joblib")
        print(f"Loading model: {model_filename} for choice {choice}")
        ensemble_model = joblib.load(model_filename)

        # Ensure input data is in the correct format
        X_modified = X_modified.astype(np.float32)

        # Predict using the loaded model
        y_pred = ensemble_model.predict(X_modified)

        # Scale predictions
        y_pred_scaled = 10**y_pred - 1

        predictions.append(y_pred_scaled)

        # Add predictions for this iteration to the DataFrame
        all_predictions_combined[f"{iteration}_{choice}"] = y_pred_scaled

    # Convert predictions to a numpy array for statistical calculations
    predictions_array = np.array(predictions)  # Shape: (30, num_samples)

    # Calculate mean and standard deviation for each sample
    mean_predictions = predictions_array.mean(axis=0)  # Mean along iterations
    std_predictions = predictions_array.std(axis=0)    # Std deviation along iterations

    # Add mean and std to the combined DataFrame
    all_predictions_combined[f"Mean_{choice}"] = mean_predictions
    all_predictions_combined[f"Std_{choice}"] = std_predictions

    corrected[f"Mean_{choice}"] = mean_predictions
    corrected[f"Std_{choice}"] = std_predictions

# Save all predictions and statistics to a single Excel file
output_file = "/predictions.xlsx"
all_predictions_combined.to_excel(output_file, index=False)

print(f"Predictions and statistics saved to {output_file}")

Surface age correction


In [ ]:
#surface age corrected (H ion) = surface age at 1 AU * (distance)^2

#For Itokawa  a = 1.3 for Eros a = 1.4
a=1.3

H_corrected = merged_df['Mean_H']*a**2
H_std_corrected = merged_df['Std_H']*a**2

merged_df['H_corrected'] = H_corrected
merged_df['H_std_corrected'] = H_std_corrected

In [ ]:
print(H_corrected)

In [ ]:
#base on Grun et. al (1991) and Divine et. al (1993) and Jehn et. al (200)

W = 10**(-12)   #weight of the particle in g
B = (math.log10(W)+11.5)/5.5
B2 = 1-B**2
gamma = (math.log10(W)+12)/6
F = 10**(-4)    # flux at 1 AU
T = 365*24*3600 #time in s
A1AU = 0.00035478

o6 = (0.138+0.142 * am+ 0.408*a**2)** - 1
o12 =(6.8-1.96 * a + 0.16 * (a**2))
frel_1 = (5**B2)*o6
frel_2 = gamma**2+(1-gamma**2)*o12
frel = frel_1/frel_2
f = F*frel  #fluence
v = 11300*a**2 - 38900*a + 42600  #velocity m/s interpotalet between values at 1 Au, 2 Au and 3 AU.

A= 0.5*W/1000*(v**2)*f*T  #1/2*particle weight*Impact average velocity*Flux*Time per year in seconds

#surface age corrected (laser) = surface age at 1 AU * A1AU/A

L_corrected = all_predictions_combined["Mean_L"]*(A1AU/A)
L_std_corrected =all_predictions_combined['Std_L']*(A1AU/A)

all_predictions_combined['L_corrected'] = L_corrected
all_predictions_combined['L_std_corrected'] = L_std_corrected

corrected['L_corrected'] = L_corrected
corrected['L_std_corrected'] = L_std_corrected


# Save to an Excel file
merged_df.to_excel('/Surface_age_IEros.xlsx', index=False) # replace with your file name